# 1. Inicializar o Dataset

## 1.1. Importar as dependências

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

## 1.2. Importar o DataFrame

In [ ]:
df = pd.read_csv("docs/heart.csv")

## 1.3. Checar como os campos foram interpretados

In [ ]:
df.head()

In [ ]:
df.dtypes

In [ ]:
df.shape

## 1.4. Tabelas da seção 3.1 do relatório

### Tabela 2

In [ ]:
numeric_cols = ["Age", "RestingBP", "Cholesterol", "FastingBS", "MaxHR", "Oldpeak"]

desc_numeric = df[numeric_cols].describe().T

desc_numeric

### Tabela 3

In [ ]:
cat_cols = ["Sex", "ChestPainType", "RestingECG", "ExerciseAngina", "ST_Slope"]

freqs = {}
for col in cat_cols:
    vc = df[col].value_counts()
    rel = df[col].value_counts(normalize=True) * 100
    freqs[col] = pd.DataFrame({
        "categoria": vc.index,
        "frequencia": vc.values,
        "percentual": rel.values.round(2)
    })

In [ ]:
freqs["Sex"]

In [ ]:
freqs["ChestPainType"]

In [ ]:
freqs["RestingECG"]

In [ ]:
freqs["ExerciseAngina"]

In [ ]:
freqs["ST_Slope"]

### Tabela 4

In [ ]:
target_counts = df["HeartDisease"].value_counts()
target_rel = df["HeartDisease"].value_counts(normalize=True) * 100

tabela4 = pd.DataFrame({
    "classe": target_counts.index,
    "frequencia": target_counts.values,
    "percentual": target_rel.values.round(2)
})

tabela4

## 1.5. Gráficos da seção 3.2 do relatório

### Histogramas (Age, RestingBP, Cholesterol, MaxHR)

In [ ]:
cols_hist = ["Age", "RestingBP", "Cholesterol", "MaxHR"]

for col in cols_hist:
    plt.figure()
    df[col].hist(bins=20)
    plt.title(f"Histograma de {col}")
    plt.xlabel(col)
    plt.ylabel("Frequência")
    plt.tight_layout()
    plt.show()


### Gráficos para os categóricos

In [ ]:
cols_bar = ["Sex", "ChestPainType", "ExerciseAngina", "ST_Slope"]

for col in cols_bar:
    plt.figure()
    df[col].value_counts().plot(kind="bar")
    plt.title(f"Distribuição de {col}")
    plt.xlabel(col)
    plt.ylabel("Frequência")
    plt.tight_layout()
    plt.show()


## 1.6. Gráfico de barras da variável alvo

In [ ]:
plt.figure()
df["HeartDisease"].value_counts().plot(kind="bar")
plt.title("Distribuição da variável alvo HeartDisease")
plt.xlabel("HeartDisease (0 = sem doença, 1 = com doença)")
plt.ylabel("Frequência")
plt.tight_layout()
plt.show()


## 1.7. Tabela da seção 3.3

In [ ]:
missing = df.isna().sum().to_frame(name="faltantes")
missing["percentual"] = (missing["faltantes"] / len(df) * 100).round(2)
missing

# 2. Pré-processamento

## 2.1. Separar atributo alvo

In [ ]:
target_col = "HeartDisease"

## 2.2. Codificar variáveis categóricas (one-hot encoding)

In [ ]:
cat_cols = ["Sex", "ChestPainType", "RestingECG", "ExerciseAngina", "ST_Slope"]

df_encoded = pd.get_dummies(df, columns=cat_cols, drop_first=True)

X = df_encoded.drop(columns=[target_col])
y = df_encoded[target_col]

## 2.3. Separar em treino e teste (70% / 30%), estratificando pela classe

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.3,
    stratify=y,
    shuffle=True,
    random_state=42,
)

## 2.4. Normalizar atributos numéricos para modelos sensíveis à escala

In [ ]:
numeric_cols = ["Age", "RestingBP", "Cholesterol", "FastingBS", "MaxHR", "Oldpeak"]

scaler = StandardScaler()

X_train_scaled = X_train.copy()
X_test_scaled = X_test.copy()

X_train_scaled[numeric_cols] = scaler.fit_transform(X_train[numeric_cols])
X_test_scaled[numeric_cols] = scaler.transform(X_test[numeric_cols])

# 3. Modelagem e avaliação

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    classification_report,
    confusion_matrix,
)

import time
import pandas as pd

In [ ]:
# Lista de modelos: (nome, instancia, usar_dados_escalados?)
modelos = [
    ("Regressão Logística", LogisticRegression(max_iter=1000, random_state=42), True),
    ("Árvore de Decisão",   DecisionTreeClassifier(random_state=42),            False),
    ("Random Forest",       RandomForestClassifier(n_estimators=200, random_state=42), False),
]

resultados = []
relatorios = {}
matrizes_confusao = {}

for nome, modelo, usar_escalado in modelos:
    if usar_escalado:
        X_tr = X_train_scaled
        X_te = X_test_scaled
    else:
        X_tr = X_train
        X_te = X_test

    # Tempo de treinamento
    inicio = time.time()
    modelo.fit(X_tr, y_train)
    fim = time.time()
    tempo_treino = fim - inicio

    # Predições
    y_pred = modelo.predict(X_te)

    # Métricas
    acc = accuracy_score(y_test, y_pred)
    prec = precision_score(y_test, y_pred, average="binary", pos_label=1)
    rec = recall_score(y_test, y_pred, average="binary", pos_label=1)
    f1 = f1_score(y_test, y_pred, average="binary", pos_label=1)

    resultados.append({
        "Modelo": nome,
        "Acurácia": acc,
        "Precisão": prec,
        "Recall": rec,
        "F1": f1,
        "Tempo_treino_seg": tempo_treino,
    })

    # Guardar relatório por classe e matriz de confusão para usar depois na Seção 6
    relatorios[nome] = classification_report(y_test, y_pred, output_dict=False)
    matrizes_confusao[nome] = confusion_matrix(y_test, y_pred)

# DataFrame com os resultados gerais
df_resultados = pd.DataFrame(resultados).sort_values(by="Acurácia", ascending=False)
df_resultados

In [ ]:
relatorios